In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.simplefilter(action='ignore', category=Warning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://www.researchgate.net/profile/Raid-Al-Ani/post/What_is_the_difference_between_Rationalism_Empiricism_give_your_opinion/attachment/59d7936df3ec044767de8d04/AS%3A546396342554624%401507282447588/image/RationalismEmpiricism.png)researchgate.net

#Rationalism

In philosophy, rationalism is the epistemological view that "regards reason as the chief source and test of knowledge" or "any view appealing to reason as a source of knowledge or justification". More formally, rationalism is defined as a methodology or a theory "in which the criterion of the truth is not sensory but intellectual and deductive.

https://en.wikipedia.org/wiki/Rationalism

In [ ]:
df=pd.read_csv('/kaggle/input/rationalist-thinkers/Rationalism_works_corpus.csv',encoding ='ISO-8859-1',sep=",")
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
import gensim

import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

#Code by Syurenuko https://www.kaggle.com/syurenuko/clrp-word2vec-lightgbm-baseline/data

In [ ]:
data_dir = '../input/rationalist-thinkers/'
sample_submission = pd.read_csv(data_dir + 'Rationalism_works_corpus.csv')
target = df['publishing_date'].to_numpy()

#Embedding by Word2vec

In [ ]:
#Code by Syurenuko https://www.kaggle.com/syurenuko/clrp-word2vec-lightgbm-baseline/data

word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('../input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', binary=True)
print(word2vec_model.vectors.shape)

In [ ]:
#Code by Syurenuko https://www.kaggle.com/syurenuko/clrp-word2vec-lightgbm-baseline/data

def avg_feature_vector(sentence, model, num_features):
    words = sentence.replace('\n'," ").replace(',',' ').replace('.'," ").split()
    feature_vec = np.zeros((num_features,),dtype="float32")#特徴ベクトルの初期化
    i=0
    for word in words:
        try:
            feature_vec = np.add(feature_vec, model[word])
        except KeyError as error:
            feature_vec 
            i = i + 1
    if len(words) > 0:
        feature_vec = np.divide(feature_vec, len(words)- i)
    return feature_vec

In [ ]:
#Code by Syurenuko https://www.kaggle.com/syurenuko/clrp-word2vec-lightgbm-baseline/data

word2vec_df = np.zeros((len(df.index),300),dtype="float32")#特徴ベクトルの初期化
#word2vec_test = np.zeros((len(test.index),300),dtype="float32")

for i in range(len(df.index)):
    word2vec_df[i] = avg_feature_vector(df["text_clean"][i],word2vec_model, 300)
    
#for i in range(len(test.index)):
 #   word2vec_test[i] = avg_feature_vector(test["text_clean"][i],word2vec_model, 300)

In [ ]:
print(word2vec_df.shape)
print(target.shape)
#print(word2vec_test.shape)

#Training & Inference

lightgbm (KFold=5)

In [ ]:
#Code by Syurenuko https://www.kaggle.com/syurenuko/clrp-word2vec-lightgbm-baseline/data

#parameter settings
params = {
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'objective': 'regression',
    'seed': 42,
    'learning_rate': 0.01,
    "n_jobs": -1,
    "verbose": -1
}

pred = np.zeros(df.shape[0])

In [ ]:
#KFold 　n_splits=5
from sklearn.model_selection import KFold
fold = KFold(n_splits=5, shuffle=True, random_state=42)
cv=list(fold.split(word2vec_df, target))

In [ ]:
#Code by Syurenuko https://www.kaggle.com/syurenuko/clrp-word2vec-lightgbm-baseline/data

rmses = []
for tr_idx, val_idx in cv: 
    x_tr, x_va = word2vec_df[tr_idx], word2vec_df[val_idx]
    y_tr, y_va = target[tr_idx], target[val_idx]
        
    df_set = lgb.Dataset(x_tr, y_tr)
    val_set = lgb.Dataset(x_va, y_va, reference=df_set)
        
    # Training
    model = lgb.train(params, df_set, num_boost_round=10000, early_stopping_rounds=100,
                      valid_sets=[df_set, val_set], verbose_eval=-1)#lgb.train is module train. Don't write df
        
    y_pred = model.predict(x_va)
    rmse =  np.sqrt(mean_squared_error(y_va, y_pred))
    rmses.append(rmse)
        
    #Inference
   # test_pred = model.predict(word2vec_test)
    #pred += test_pred / 5  
        
print("\n", "Mean Fold RMSE:", np.mean(rmses))

In [ ]:
#Saving for the next competition

sample_submission.target = pred
sample_submission.to_csv('Rationalism_works_corpus.csv',index=False)

In [ ]:
#sample_submission

In [ ]:
#2nd row. And 6th column, text_clean 

df.iloc[1,5]

#Thanks to Syurenuko https://www.kaggle.com/syurenuko/clrp-word2vec-lightgbm-baseline/data

Again.
